# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>
# <center>2.1 Experimental notebook - Parsing</center>

---

This is the ``Experimental`` ``notebook`` ``2.1 "Parsing"``, where all the relevent data of the experiments are read in from different ressources. For each individual experiment this workflow is to be executed once, and the data can be appended to the project's dataset.

---

---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [1]:
# Activate autoreload to keep on track with changing modules #
%reload_ext autoreload
%autoreload 2

# Import standard libraries #
import os
import json
import logging
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, Markdown

# Import librarian module for file directory handling #
from datamodel_b07_tc.tools import Librarian

# Import modified sdRDM objects #
from datamodel_b07_tc.modified.experiment import Experiment
from datamodel_b07_tc.modified.measurement import Measurement
from datamodel_b07_tc.modified.plantsetup import PlantSetup

# Import datamodel from sdRDM #
from sdRDM import DataModel

# Import tools for parsing and calibration of the raw data #
from datamodel_b07_tc.tools import Calibrator
from datamodel_b07_tc.tools import gc_parser
from datamodel_b07_tc.tools import gstatic_parser
from datamodel_b07_tc.tools import mfm_parser
# from datamodel_b07_tc.tools import DEXPI2sdRDM

# from sdRDM.generator import generate_python_api
# generate_python_api('specifications/datamodel_b07_tc.md', '', 'datamodel_b07_tc')

In [4]:
#Define paths for loggin output #
root                = Path.cwd()
logging_config_path = root / "datamodel_b07_tc/tools/logging/config_exp_2_1.json"

# Read in logger specs and configurate logger (set name to current notebook) #
with open(logging_config_path) as logging_config_json: logging.config.dictConfig( json.load( logging_config_json ) )
logger = logging.getLogger(__name__)

# Set the level of thid-party logger to avoid dumping too much information #
third_party_module_loggers = ['markdown_it', 'h5py', 'numexpr', 'git']
for logger_ in third_party_module_loggers: logging.getLogger(logger_).setLevel('WARNING')

# Initialize the librarian with root directory of this notebook #
librarian = Librarian(root_directory=root)

---
## Section 1: Dataset and data model parsing
---
In this section the data model and the dataset as well as all the output files necessary for the analysis notenook are parsed.  

In [2]:

def search_files_in_subdirectory(root_directory: Path, directory_keys: list[str], file_filter: str, verbose: bool = None) -> Path:
    """
    Function that loobs through Path objects containing a main directory. In this directory it is recoursevly searched for sub directories. 
    In the last sub directory files with the suffix 'file_filter' are searched and returned

    Args:
        root_directory (Path): Root directory
        directory_keys (list[str]): List of subdirectories that should be recoursevly searched
        file_filter (str): Suffix of files that should be found in last given sub directory
        verbose (bool, optional): Possiblity to printout all subdirectories in each directory listed. Defaults to None.

    Raises:
        KeyError: If either the specified sub directory or file could not be found

    Returns:
        subdirectory_files (Path): Path object containing all files found in the subdirectory
    """

    # First search for every nested sub directory in provided root directory #
    root = root_directory
    for j,directory_key in enumerate(directory_keys):
        try:
            idx_sub_directory = [i for i in range(len(root)) if root[i].parts[-1] == directory_key ][0]
            if j < len(directory_keys)-1: 
                root          = librarian.enumerate_subdirectories(directory=root[idx_sub_directory])
        except:
            raise KeyError("Defined key: '%s' cannot be found in the given root directory: %s"%(directory_key,root[0].parent))

    # Search for all files that match the given filter in the specified sub directory #
    subdirectory_files = librarian.enumerate_files(directory=root[idx_sub_directory], filter=file_filter, verbose=verbose)   
    if not bool(subdirectory_files): 
        raise KeyError("No files with filter: '%s' found in the given sub directory: %s"%(file_filter,root_directory[idx_sub_directory]))
    
    return subdirectory_files


def dropdown_files(files: Path, description: str = "Files"):
     
    dropdown = widgets.Dropdown(
        options=[(path.parts[-1],idx) for idx,path in files.items()],
        description=description,
        layout=widgets.Layout(width='auto'),
        style={'description_width': 'auto'} 
        )
    
    return dropdown

In [5]:
# Check for all available subdirectories #
root                          = Path.cwd()
root_subdirectories           = librarian.enumerate_subdirectories(directory=root)

# Search for subdirectory "datasets" and in it for all dataset json files #

json_dataset_files            = search_files_in_subdirectory(root_directory=root_subdirectories, directory_keys=["datasets"], file_filter="json", verbose=False)
json_dropdown                 = dropdown_files(json_dataset_files,description="Datasets")

dataset, lib = DataModel.parse( json_dataset_files[ json_dropdown.value ] )

## Search for raw data ##

# Potentiostatic data #
potentiostatic_raw_data_files = search_files_in_subdirectory(root_directory=root_subdirectories, directory_keys=["data","Rohdaten","01_EChem","CAD14-Cu@AB"], file_filter="DTA", verbose=False)
potentiostatic_dropdown       = dropdown_files(potentiostatic_raw_data_files,description="Potentiostatic raw data")

# Mass flow meter data #
mfm_raw_data_files            = search_files_in_subdirectory(root_directory=root_subdirectories, directory_keys=["data","Rohdaten","03_MFM","CAD14-Cu@AB"], file_filter="csv", verbose=False)
mfm_dropdown                  = dropdown_files(mfm_raw_data_files,description="Mass flow meter raw data")

# GC data #


display( Markdown("### Choose a dataset") )
display( json_dropdown )

display( Markdown("### Choose raw data files") )

display( widgets.HBox([ potentiostatic_dropdown, mfm_dropdown ]) )

# Read in selected raw data

#potentiostatic_metadata_df, potentiostatic_measurement = Measurement.from_parser( parser=gstatic_parser, metadata_path=potentiostatic_raw_data_files[ potentiostatic_dropdown.value ] )
#mfm_experimental_data_df, mfm_measurement = Measurement.from_parser( parser=mfm_parser, experimental_data_path=mfm_raw_data_files[ mfm_dropdown.value ] )

### Choose a dataset

Dropdown(description='Datasets', layout=Layout(width='auto'), options=(('b07.json', 0),), style=DescriptionSty…

### Choose raw data files

In [58]:

folders = root_subdirectories

class widget():
    
    # Function to navigate into the selected subfolder

    def go_to_subfolder(self,_):
        self.current_dir.value       = str(self.folder_dropdown.value)
        subfolders                   = librarian.enumerate_subdirectories(directory=self.folder_dropdown.value)
        self.folder_dropdown.options = [ (path.parts[-1],path) for _,path in subfolders.items() ] if bool(subfolders) else [ ("No subdirectories",self.folder_dropdown.value) ]

    # Function to navigate back from the selected subfolder
    def go_to_parentfolder(self,_):
        parentfolder                 = self.parent
        self.current_dir.value       = str(parentfolder)
        parentfolders                = librarian.enumerate_subdirectories(directory=parentfolder)
        self.folder_dropdown.options = [(path.parts[-1],path) for _,path in parentfolders.items()]
    
    def add_file(self,_):
        if self.file_category.value == "EChem":
            self.Echem_list.append(str( self.file_dropdown.value.parts[-1] ))
            self.Echem_files.value = self.Echem_list

        elif self.file_category.value == "GC":
            self.GC_list.append(str( self.file_dropdown.value.parts[-1] ))
            self.GC_files.value    = self.GC_list

        elif self.file_category.value == "MFM":
            self.MFM_list.append(str( self.file_dropdown.value.parts[-1] ))
            self.MFM_files.value  = self.MFM_list

    def folder_dropdown_option_handler(self,_):
        # If no subdirectories exist, then the parent folder is simply the first parent, otherwise it is the 2nd parent
        # (because the current dropdown value is already )
        if str(self.folder_dropdown.value.parent) == self.current_dir.value:
            self.parent                  = self.folder_dropdown.value.parent.parent
            self.file_folder             = self.folder_dropdown.value.parent
        else:
            self.parent                  = self.folder_dropdown.value.parent
            self.file_folder             = self.folder_dropdown.value

        # Reset file type after chaning dropdown

        self.file_type_text.value        = ""
        self.file_dropdown.options       = []

    def file_type_input_handler(self,_):
        if self.file_type_text.value:
            file_filter                  = self.file_type_text.value
            subdirectory_files           = librarian.enumerate_files(directory=self.file_folder, filter=file_filter)
            
            # Show all available files and show the first initially that they know if file are available
            try:
                self.file_dropdown.options   = [(file.parts[-1],file) for _,file in subdirectory_files.items()]
                self.file_dropdown.value     = subdirectory_files[0]
            except:
                self.file_dropdown.options   = ["No files with specified suffix"]
                self.file_dropdown.value     = "No files with specified suffix"
    
    def file_category_input_handler(self,_):
        self.button_select.description = 'Add file to %s'%(self.file_category.value)
        
    
    def visualize_folderwidget(self,folders: Path) -> None:
        
        self.folder_dropdown = widgets.Dropdown(description='Select folder:',
                                                options=[(path.parts[-1],path) for _,path in folders.items()],
                                                layout=widgets.Layout(width='auto'),
                                                style={'description_width': 'auto'})

        self.file_dropdown   = widgets.Dropdown(description='Select file:',
                                                layout=widgets.Layout(width='auto'),
                                                style={'description_width': 'auto'})
        
        self.file_category   = widgets.Dropdown(options=['EChem', 'GC', 'MFM'],
                                                value='EChem',
                                                description='for category:',
                                                style={'description_width': 'auto'})

        self.button_go_for   = widgets.Button(description='Move into directory',
                                              layout=widgets.Layout(width='auto'))
        
        self.button_go_back  = widgets.Button(description='Move one diretory back',
                                              layout=widgets.Layout(width='auto'))
        
        self.button_select   = widgets.Button(description='Add file to %s'%(self.file_category.value),
                                              layout=widgets.Layout(width='auto'))

        self.file_type_text  = widgets.Text(description='File type:',
                                            placeholder='Enter type here (e.g.: csv, json, ... or * for all files)',
                                            layout=widgets.Layout(width='auto'),
                                            style={'description_width': 'auto'})
        
        self.current_dir     = widgets.Text(description='Current directory:',
                                            disabled=True,
                                            value=str(folders[0].parent),
                                            layout=widgets.Layout(width='auto'),
                                            style={'description_width': 'auto'})

        self.Echem_files     = widgets.TagsInput(allow_duplicates=False,disabled=True)
        self.GC_files        = widgets.TagsInput(allow_duplicates=False,disabled=True)
        self.MFM_files       = widgets.TagsInput(allow_duplicates=False,disabled=True)

        self.file_folder     = folders[0].parent
        self.Echem_list      = []
        self.GC_list         = []
        self.MFM_list        = []
        
        # --> function that shows and store the selected files --> either fix that GC, EChem or MFM data
        self.select_file     = 5
        
        # Functions for the buttons #
        self.button_go_for.on_click(self.go_to_subfolder)
        self.button_go_back.on_click(self.go_to_parentfolder)
        self.button_select.on_click(self.add_file)

        # Attach the event handler to the 'value' property change of the file type widget
        self.file_type_text.observe(self.file_type_input_handler, names='value')
        self.folder_dropdown.observe(self.folder_dropdown_option_handler, names='options')
        self.file_category.observe(self.file_category_input_handler, names='value')

        # Display the widgets

        # Create the layout
        widgets1 = widgets.VBox([self.current_dir,self.folder_dropdown])
        widgets2 = widgets.HBox([self.button_go_for, self.button_go_back])
        widgets3 = widgets.VBox([self.file_type_text])
        widgets4 = widgets.HBox([self.file_dropdown,self.file_category])
        widgets5 = widgets.VBox([self.button_select])
        widgets6 = widgets.HBox([widgets.VBox([widgets.Label(value='Files for EChem evaluation:'), self.Echem_files]),
                                 widgets.VBox([widgets.Label(value='Files for GC evaluation:'), self.GC_files]),
                                 widgets.VBox([widgets.Label(value='Files for MFM evaluation:'), self.MFM_files])])

        # Combine the layout
        full_layout = widgets.VBox([widgets1,widgets2,widgets3,widgets4,widgets5,widgets6])

        # Display the layout
        display(full_layout)



In [59]:
w = widget()
w.visualize_folderwidget(folders)

In [80]:
# Search for mass flow meter data folder #
idx_gc_direct  = [i for i in range(len(raw_data_subdirectories)) if str(raw_data_subdirectories[i]).split("/")[-1] == "02_GC" ][0]
gc_directories = librarian.enumerate_subdirectories(directory=raw_data_subdirectories[idx_gc_direct])

# Serach for gc subdirectories #
idx_gc_sub_dir    = [i for i in range(len(gc_directories)) if str(gc_directories[i]).split("/")[-1] == "CAD14-Cu@AB" ][0]
gc_subdirectories = librarian.enumerate_subdirectories(directory=gc_directories[idx_gc_sub_dir])

gc_subdirectories

IndexError: list index out of range

In [ ]:
# Search for mass flow meter data folder #
idx_gc_direct  = [i for i in range(len(raw_data_subdirectories)) if str(raw_data_subdirectories[i]).split("/")[-1] == "02_GC" ]
gc_directories = librarian.enumerate_subdirectories(directory=raw_data_subdirectories[idx_gc_direct])

# Serach for gc subdirectories #
idx_gc_sub_dir    = [i for i in range(len(gc_directories)) if str(gc_directories[i]).split("/")[-1] == "CAD14-Cu@AB" ]
gc_subdirectories = librarian.enumerate_subdirectories(directory=gc_directories[idx_gc_sub_dir])

# Select subdirectory of wanted experiment from given directories #
idx_gc_sub_sub_dir   = 0
gc_subsubdirectories = librarian.enumerate_subdirectories(directory=gc_subdirectories[idx_gc_sub_sub_dir])

# Gather all the gc raw data files #
gc_raw_data_files_list = []

# Select the indices of the subdirectories that should be read in
gc_raw_data_subdir_idx = [ 3, 4, 5 ] 

gc_raw_data_files_list = [librarian.enumerate_files(directory=gc_subsubdirectories[i], filter='CSV') for i in gc_raw_data_subdir_idx]

# Read out all the data from the provided gc files #
gc_experimental_data_df_list = []
gc_metadata_df_list = []
gc_measurements_list = []

for data_file in gc_raw_data_files_list:
    gc_metadata_df, gc_experimental_data_df, gc_measurement = Measurement.from_parser(
        parser=gc_parser,
        metadata_path=data_file[0],
        experimental_data_path=data_file[1]
    )
    gc_experimental_data_df_list.append(gc_experimental_data_df)
    gc_metadata_df_list.append(gc_metadata_df)
    gc_measurements_list.append(gc_measurement)

---
## Section 2: Plant setup parsing
---

Instantiate 'experiment' object.

In [22]:
experiment = Experiment()

## Get plan setup given as in dexpi format ##

#idx_rawdatafolder       = [i for i in range(len(data_subdirectories)) if str(data_subdirectories[i]).split("/")[-1] == "plant_setup" ]
#plant_setup_files = librarian.enumerate_files(dirctory=data_subdirectories[idx_rawdatafolder], filter='xml')
#plant_setup = PlantSetup.from_parser(parser=DEXPI2sdRDM, path=plant_setup_files[0])
#experiment.plant_setup = plant_setup

---
## Section 3: Potenstiostatic data parsing
---
Select path to the potentiostatic data and print available subdirectories.

---
## Section 4: MFM data parsing
---
Provide name of the subdirectory containing the mass flow meter measurement data.

---
## Section 5: GC data parsing
---

Parent directory: 
 /mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/Rohdaten/02_GC 
Available subdirectories:
0: .../CAD14-Cu@AB


In [ ]:
## Combine all gathered data in the experiments object ##

experiment.measurements = [potentiostatic_measurement, mfm_measurement, *gc_measurements_list]

---
## Section 6: Calibration data parsing
---

Search for calibation files in the 'calibration' directory.

In [42]:
calibration_files = librarian.enumerate_files(directory=data_subdirectories[0])

Directory: 
 /mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/calibration 
Available files:
0: calibration.json


Initialize calibrator with an available calibration file selected by its index.

In [43]:
calibrator = Calibrator.from_json_file(path_to_json_file=calibration_files[0])

Calibrate and return analysis object with calibration parameters just computed. <br> Append the resulting SpeciesData objects to the experiment object.

In [44]:
species_data_list = calibrator.calibrate()
experiment.species_data = species_data_list

---
## Section 7: Parsing auxiliary data
---

### Correction factors

Search for correction factors files in the 'correction factors' directory.

In [45]:
correction_factors_files = librarian.enumerate_files(directory=data_subdirectories[1])

Directory: 
 /mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/correction_factors 
Available files:
0: correction_factors.json


Load correction factors into the experiment object.

In [46]:
experiment.read_correction_factors(correction_factors_files[0])

### Farady coefficients

Search for faraday coefficients files in the 'correction factors' directory.

In [47]:
faraday_coefficients_files = librarian.enumerate_files(directory=data_subdirectories[2])

Directory: 
 /mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/data/faraday_coefficients 
Available files:
0: faraday_coefficients.json


Load faraday coefficients into the experiment object.

In [48]:
experiment.read_faraday_coefficients(faraday_coefficients_files[0])

### Electrode surface area

Set value for the surface area of the electrode.

In [49]:
electrode_surface_area = 1.0 # cm^2

---
## Section 8: Appending parsed data to dataset
---

Print current state of experiment object.

In [50]:
# print(experiment.json())

Append experiment object to the dataset.

In [51]:
dataset.experiments.append(experiment)

Replace 'old' dataset by its extended version containing all the parsef data.

In [52]:
with open(json_dataset, "w") as f:
    f.write(dataset.json())